In [3]:
import nest_asyncio
nest_asyncio.apply()

import twint

c = twint.Config()

c.Search = ['#jamescharles']
c.Limit = 50000
c.Store_csv = True
c.Since = '2019-01-01'
c.Output = "jamescharles.csv"

twint.run.Search(c)

ModuleNotFoundError: No module named 'twint'